# Libraries load 

In [12]:
import json
from scipy.optimize import newton_krylov
from pyomo.environ import SolverFactory, SolverManagerFactory
from pyomo.environ import*
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pulp import LpVariable, LpMinimize, LpProblem, lpSum, value, GUROBI, GLPK_CMD
import envs as en
import pyomo.contrib.solver.ipopt
import os
from pyomo.environ import value

from io import BytesIO
from typing import List, Optional, Dict, Any
from pydantic import BaseModel

import requests

# Input file load


In [13]:
""" input_file = "DataSets/V3/sampledata4.xlsx"

# Spanish prosumer comunity example
prosumer_id_mapping = {
        0: "abc1",
        1: "mnl2",
        2: "xyz3",
        3: "def4",
        4: "ghi5"
    } """

' input_file = "DataSets/V3/sampledata4.xlsx"\n\n# Spanish prosumer comunity example\nprosumer_id_mapping = {\n        0: "abc1",\n        1: "mnl2",\n        2: "xyz3",\n        3: "def4",\n        4: "ghi5"\n    } '

# Code Methods


## Active atributes of a prosumer

Contains a string which represents the prosumer identification and is associated with a key (integer number), and a boolean for each column of data that is activated (true / value =! 0) or deactivated (false/value==0).


In [14]:
class ActiveAttribute(BaseModel):
    prosumerId: str
    profileLoad: bool
    stateOfCharge: bool
    photovoltaicEnergyLoad: bool

## Load excel file

    Load all sheets from an Excel file into a dictionary of DataFrames.
    
    Args:
        file_path (str): Path to the Excel file.
        
    Returns:
        Dict[str, pd.DataFrame]: Dictionary mapping sheet names to their DataFrames.
        
    Raises:
        FileNotFoundError: If the file does not exist.
        ValueError: If the file cannot be read or required sheets are missing.

In [ ]:
def load_excel_file(file_path: str) -> Dict[str, pd.DataFrame]:

    if not os.path.exists(file_path):
        raise FileNotFoundError(f"Excel file not found at: {file_path}")
    
    try:
        # Load all sheets
        xl = pd.read_excel(file_path, sheet_name=None, engine='openpyxl')
        required_sheets = ['PL', 'PPV_capacity']
        for sheet in required_sheets:
            if sheet not in xl:
                raise ValueError(f"Required sheet '{sheet}' not found in the Excel file.")
        return xl
    except Exception as e:
        raise ValueError(f"Failed to read Excel file: {str(e)}")

: 

In [ ]:
""" def load_excel_file(file_path: str) -> Dict[str, pd.DataFrame]:
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"Excel file not found at: {file_path}")
    
    try:
        # Load all sheets
        xl = pd.read_excel(file_path, sheet_name=None, engine='openpyxl')

        # Validar existência das sheets obrigatórias
        required_sheets = ['PL', 'PPV_capacity']
        for sheet in required_sheets:
            if sheet not in xl:
                raise ValueError(f"Required sheet '{sheet}' not found in the Excel file.")
        
        # Validar colunas obrigatórias por sheet
        required_columns = {
            'PPV_capacity': ['Datetime', 'Value'],  # ajusta conforme o teu ficheiro
            'PL': ['Datetime', 'PLValue']           # ajusta conforme o teu ficheiro
        }

        for sheet_name, expected_columns in required_columns.items():
            df = xl[sheet_name]
            missing_cols = [col for col in expected_columns if col not in df.columns]
            if missing_cols:
                raise ValueError(
                    f"Sheet '{sheet_name}' is missing required columns: {', '.join(missing_cols)}"
                )

        return xl

    except Exception as e:
        raise ValueError(f"Failed to read Excel file: {str(e)}") """

## Print DataFrame

    Prints each DataFram within a dictionary, identifying by the name of the key.
    

In [16]:
from typing import Dict
import pandas as pd

def print_dataframes_dict(dataframes: Dict[str, pd.DataFrame]):
    for name, df in dataframes.items():
        print(f"\n=== DataFrame: {name} ===")
        print(df.head())  # ou apenas print(df) se quiser tudo


## Check zero columns

    Check which columns in the specified DataFrame have all values equal to zero.
    
    Args:
        df (pd.DataFrame): DataFrame of the sheet to check.
        sheet_name (str): Name of the sheet ('PL' or 'PPV_capacity').
        
    Returns:
        List[str]: List of column names with all values equal to zero.
        
    Raises:
        ValueError: If the sheet name is invalid or columns are not found.

In [17]:
def check_zero_columns(df: pd.DataFrame, sheet_name: str) -> List[str]:

    valid_columns = {
        'PL': [col for col in df.columns if col.startswith('PL')],
        'PPV_capacity': [col for col in df.columns if col.startswith('PV')]
    }
    
    if sheet_name not in valid_columns:
        raise ValueError(f"Invalid sheet name: {sheet_name}. Must be 'PL' or 'PPV_capacity'.")
    
    zero_columns = []
    for col in valid_columns[sheet_name]:
        if col in df.columns and (df[col] == 0).all():
            zero_columns.append(col)
    
    return zero_columns

## Set Column to zero 

    Set all values in the specified column of the DataFrame to zero.
    
    Args:
        df (pd.DataFrame): DataFrame of the sheet to modify.
        sheet_name (str): Name of the sheet ('PL' or 'PPV_capacity').
        column_name (str): Name of the column to set to zero (e.g., 'PL1', 'PV2').
        
    Returns:
        pd.DataFrame: Modified DataFrame with the specified column set to zero.
        
    Raises:
        ValueError: If the sheet name or column name is invalid.

In [18]:
def set_column_to_zero(df: pd.DataFrame, sheet_name: str, column_name: str) -> pd.DataFrame:

    valid_columns = {
        'PL': [col for col in df.columns if col.startswith('PL')],
        'PPV_capacity': [col for col in df.columns if col.startswith('PV')]
    }
    
    if sheet_name not in valid_columns:
        raise ValueError(f"Invalid sheet name: {sheet_name}. Must be 'PL' or 'PPV_capacity'.")
    
    if column_name not in valid_columns[sheet_name]:
        raise ValueError(f"Invalid column name: {column_name}. Must be one of {valid_columns[sheet_name]}.")
    
    if column_name not in df.columns:
        raise ValueError(f"Column {column_name} not found in sheet {sheet_name}.")
    
    df[column_name] = 0
    return df

## Save excel file 

    Save the modified DataFrames back to an Excel file.
    
    Args:
        sheets (Dict[str, pd.DataFrame]): Dictionary of sheet names and their DataFrames.
        Salva os DataFrames em diferentes folhas de um ficheiro Excel em memória e retorna um BytesIO.
    
        
    Raises:
        ValueError: If saving the file fails.


In [19]:
def save_excel_file(sheets: Dict[str, pd.DataFrame]) -> BytesIO:
    output = BytesIO()
    try:
        with pd.ExcelWriter(output, engine='openpyxl') as writer:
            for sheet_name, df in sheets.items():
                df.to_excel(writer, sheet_name=sheet_name, index=False)
        output.seek(0)  # Reposicionar o ponteiro no início do buffer
        return output
    except Exception as e:
        raise ValueError(f"Failed to save Excel file to memory: {str(e)}")

## Generate active atributes
  
     Update ActiveAttribute flags based on zero-columns in PL and PPV_capacity sheets,
        and zero out columns for already inactive attributes.
    
    Args:
        sheets (Dict[str, pd.DataFrame]): Dictionary of sheet names and their DataFrames.
        prosumer_id_mapping (Dict[int, str]): Mapping of integer indices (e.g., 0, 1) to prosumer IDs (e.g., 'abc1', 'mnl2').
        
    Returns:
        List[ActiveAttribute]: List of ActiveAttribute objects with updated profileLoad and photovoltaicEnergyLoad.
        
    Raises:
        ValueError: If required sheets are missing or invalid.
   

In [20]:

def generate_active_attributes(sheets: Dict[str, pd.DataFrame], active_attributes: List[ActiveAttribute]) -> List[ActiveAttribute]:

    df_pl = sheets["PL"]
    df_pv = sheets["PPV_capacity"]

    # Step 1: Update attribute flags based on 100% zero columns
    zero_pl_columns = check_zero_columns(df_pl, "PL")  # e.g., ['PL2']
    zero_pv_columns = check_zero_columns(df_pv, "PPV_capacity")  # e.g., ['PV4']

    for col in zero_pl_columns:
        idx = int(col.replace("PL", ""))
        if 0 <= idx < len(active_attributes):
            active_attributes[idx].profileLoad = False

    for col in zero_pv_columns:
        idx = int(col.replace("PV", ""))
        if 0 <= idx < len(active_attributes):
            active_attributes[idx].photovoltaicEnergyLoad = False

    # Step 2: For attributes already inactive, set corresponding columns to 0
    for idx, attr in enumerate(active_attributes):
        col_pl = f"PL{idx}"
        col_pv = f"PV{idx}"

        if not attr.profileLoad and col_pl in df_pl.columns:
            df_pl = set_column_to_zero(df_pl, "PL", col_pl)

        if not attr.photovoltaicEnergyLoad and col_pv in df_pv.columns:
            df_pv = set_column_to_zero(df_pv, "PPV_capacity", col_pv)

    # Update sheets with modified DataFrames
    sheets["PL"] = df_pl
    sheets["PPV_capacity"] = df_pv

    return active_attributes

## Main

In [21]:
def main():
    file_path = "sampledata4.xlsx"  # path to your Excel file
    sheets = load_excel_file(file_path)

    # Create example ActiveAttribute list (should match number of PLx / PVx columns)
    active_attributes = [
        ActiveAttribute(prosumerId=f"prosumer_{i}", profileLoad=True, stateOfCharge=True, photovoltaicEnergyLoad=True)
        for i in range(5)  # adjust to match your Excel columns
    ]

    active_attributes[1].profileLoad = False  # Example modification to one attribute

    updated_attributes = generate_active_attributes(sheets, active_attributes)

    # Show updated attributes
    for attr in updated_attributes:
        print(attr)

    # Save updated sheets to BytesIO
    output = save_excel_file(sheets)

    # Optional: save to disk
    with open("updated_output.xlsx", "wb") as f:
        f.write(output.read())

if __name__ == "__main__":
    main()

prosumerId='prosumer_0' profileLoad=True stateOfCharge=True photovoltaicEnergyLoad=True
prosumerId='prosumer_1' profileLoad=False stateOfCharge=True photovoltaicEnergyLoad=True
prosumerId='prosumer_2' profileLoad=True stateOfCharge=True photovoltaicEnergyLoad=True
prosumerId='prosumer_3' profileLoad=True stateOfCharge=True photovoltaicEnergyLoad=False
prosumerId='prosumer_4' profileLoad=True stateOfCharge=True photovoltaicEnergyLoad=False
